In [1]:
import pandas as pd
import numpy as np
import glob
import zipfile
import os

In [2]:
def get_availabilities(files):
    
    availability_factors = pd.DataFrame()

    for file in files:

        info = pd.read_csv(file, index_col=0, header=1).iloc[:3,0]

        availability_factors_zone = pd.read_csv(file, index_col=[0,1], header=10).reset_index(drop=True).stack()

        availability_factors_zone.name = (info["Target Year"], info["PECD Zone"])

        availability_factors = pd.concat([availability_factors, availability_factors_zone], axis=1)

    availability_factors = availability_factors.T.set_index((i for i in availability_factors.columns)).T.stack([0,1])

    availability_factors.index.names = ["hour", "climate year", "target year", "zone"]

    return availability_factors

In [3]:
with zipfile.ZipFile("data/pecd.zip") as zip_f:
    zip_f.extractall("data/")
    

In [4]:
save_hdf = "resources/res_profile.h5"

In [5]:
all_files = glob.glob("data/PECD/*")

In [6]:
onshore = [i for i in all_files if "Onshore" in i]

In [7]:
offshore = [i for i in all_files if "Offshore" in i]

In [8]:
solar = [i for i in all_files if "SolarPV_" in i]

In [9]:
csp = [i for i in all_files if "CSP_no" in i and "pre" not in i]

In [10]:
missing_csp = ["GR03", "PT00"]

csp = csp + [i for i in all_files if "CSP_withStorage_7h_preDispatch" in i and any(zone in i for zone in missing_csp)]

In [11]:
onshore_availabilities = get_availabilities(onshore)

In [12]:
solar_availabilities = get_availabilities(solar)

In [13]:
offshore_availabilities = get_availabilities(offshore)

In [14]:
csp_availabilities = get_availabilities(csp)

In [15]:
dirname = os.path.dirname(save_hdf)

if not os.path.exists(dirname):
    os.mkdir(dirname)

In [16]:
onshore_availabilities.to_hdf(save_hdf, "onshore")

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/tables/attributeset.py:457: NaturalNameWarning: object name is not a valid Python identifier: 'index_nameclimate year'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/tables/attributeset.py:457: NaturalNameWarning: object name is not a valid Python identifier: 'index_nametarget year'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [17]:
offshore_availabilities.to_hdf(save_hdf, "offshore")

In [18]:
solar_availabilities.to_hdf(save_hdf, "solar")

In [19]:
csp_availabilities.to_hdf(save_hdf, "CSP")

In [20]:
csp_availabilities

hour  climate year  target year  zone
0     1982          2025         CY00    0.0
                                 ES00    0.0
                                 GR03    0.0
                                 ITS1    0.0
                                 ITSI    0.0
                                        ... 
8759  2019          2033         GR03    0.0
                                 ITS1    0.0
                                 ITSI    0.0
                                 LV00    0.0
                                 PT00    0.0
Length: 9285600, dtype: float64